In [1]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
n_gpu_layers = 5  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [3]:
_PATH_LLAMA_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Llama-2-7b-Chat-GGUF/llama-2-7b-chat.Q5_K_M.gguf"
_PATH_MISTRAL_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_K_M.gguf"
_PATH_ORCA_MISTRAL_ = "/Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-OpenOrca-GGUF/mistral-7b-openorca.Q5_K_M.gguf"

In [4]:
MISTRAL_ORCA = LlamaCpp(
    model_path=_PATH_ORCA_MISTRAL_,
    n_gpu_layers=n_gpu_layers,
    n_batch=64,
    n_ctx=4096,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
    max_tokens=2000)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/niteshkumarsharma/.cache/lm-studio/models/TheBloke/Mistral-7B-OpenOrca-GGUF/mistral-7b-openorca.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: 

In [5]:
MISTRAL_ORCA.client.verbose = False

In [6]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [7]:
## Creating a list of few Shot Examples ##
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"}
]

In [8]:
## Specify template to format this examples ##

example_formatter_template = """ 
Word: {word}
Antonym: {antonym}\n
"""

In [9]:
example_prompt = PromptTemplate(input_variables=["word", "antonym"],
                                template=example_formatter_template)

In [10]:
example_prompt

PromptTemplate(input_variables=['word', 'antonym'], template=' \nWord: {word}\nAntonym: {antonym}\n\n')

In [11]:
few_shot_prompt = FewShotPromptTemplate(
    ## These are the examples we want to insert into prompt.
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym",
    input_variables=["input"],
    example_separator="\n\n"
)

In [12]:
print(few_shot_prompt.format(input="big"))

Give the antonym of every input

 
Word: happy
Antonym: sad



 
Word: tall
Antonym: short



Word: big
Antonym


## LangChain Tools & Chains

In [13]:
from langchain.chains import LLMChain

In [14]:
PROFILE_SUMMARY = """
Hello, I am Nitesh, an Experienced Machine Learning professional with 4+ years of industry expertise in leveraging advanced statistical analysis& machine learning and data analysis to lead data driven decisions

With a Master's degree from IIT Madras, I have a solid understanding of statistics, machine learning, signal processing and time series analysis.

Achievement
    Led development of a scalable & efficient Big Data ML product, which consumes close to 10TBs of data using AWS 
    Built Scalable S3 Prefixes Validation Routine using Apache Spark, achieving performance of 1M Prefixes Validation in 2 minutes using 5 Node Cluster
    Improved Logistics Business Process by Accurate Forecasting Models resulting in value creation of 1.5M USD

"""

In [48]:
## Prompt Defination
skills_template = """Extract Key Skills from Job Description Provided so that is passes the ATS. Read Job Description carefully and Give each skill a number and keep them single worded :

Job Description
"{job_desc}"
"""

In [49]:
skill_extract_prompt = PromptTemplate(
    input_variables=["job_desc"],
    template=skills_template
)

In [57]:
jd_description = """
Job Description

    Identify new opportunities to improve business processes and improve consumer experiences, and prototype solutions to demonstrate value with a crawl, walk, run mindset. 
    Work with data scientists and analysts to create and deploy new product features on the ecommerce website, in-store portals and the Levi's mobile app. 
    Implement end-to-end solutions across the full breadth of ML model development lifecycle. The specific role includes working hand in hand with the scientists from the point of data exploration for model development to the point of building features, ml pipelines and deploying them in production. You will have an opportunity to work on both batch and real time models. The role also involves operational support. 
    Establish scalable, efficient, automated processes for data analyses, model development, validation, and implementation. 
    Write efficient and scalable software to ship products in an iterative, continual release environment 
    Contribute to and promote good software engineering practices across the team and build cloud native software for ML pipelines. 
    Contribute to and re-use community best practices. 
    Embody the values and passions that characterize Levi Strauss & Co., with empathy to engage with colleagues from multiple backgrounds. 

Example Projects 

Besides Driving The Transformation Of Levi's Into a Data-driven Enterprise In General, Here Are Some Specific Projects You Will Work On And Contribute To

    Personalized in-session product recommendation engine. 
    Customer Segmentation 
    Automated text summarization and clustering 
    Next-Best offer prediction 
    Design Micro assortments for Next-Gen stores 
    Anomaly detection and Root Cause Analysis 
    Unified consumer profile with probabilistic record linkage 
    Visual search for similar and complementary products 

Who We Seek 

    Team player, solution-driven and open-minded, with a passion for continuous improvement and automation 
    Proactive and action-oriented 
    A commitment to continuous learning and knowledge sharing. 
    Ability to think logically and keep solutions simple. 
    Excellent English communication skills to collaborate effectively with cross-functional teams. 

 

Your Skills 

    Key Tech Skills: 
    Proficiency in Google Cloud Platform (GCP) tools including Vertex AI, BigQuery, Dataproc, GKE, and GCS 
    Working experience with native ML orchestration systems such as Kubeflow, Step Functions, MLflow, Airflow, TFX... 
    Experience in ETL processes 
    Proficiency in SQL 
    Familiarity with GIT for version control 
    Good understanding of Machine Learning concepts 
    Experience with object-oriented/object function scripting languages: Python, Java, C++, Scala, etc. 
    Previous experience developing predictive models in a production environment, MLOps and model integration into larger scale applications. 
    Knowledge of data pipeline and workflow management tools 
    Expertise in standard software engineering methodology, e.g. unit testing, test automation, continuous integration, code reviews, design documentation 
    Familiar with Spark 

Degree: University or advanced degree in engineering, computer science, mathematics, or a related field

Experience: Minimum 3/4 years of experience in Machine Learning Engineer role 

 

    Desired Tech Skills: 
    Software Engineering skills 
    Experience with Container Pods using Docker and Kubernetes 
    Familiarity with Big Data concepts and tools 
    Spark optimization 
    Code optimization 
    Vertex AI expertise 
    Prior experience with Jenkins scripting 
    Good-to-Have Tech Skills: 
    Agile development methodology knowledge 
    Familiarity with Data science concepts 
    API development experience 
     



"""

In [58]:
skill_extract_chain = LLMChain(llm=MISTRAL_ORCA, prompt=skill_extract_prompt, output_key="skills")

In [59]:
recruitor_template = """You are a recruitor. Take the following list of Key Skills {skills} & Current Resume Summary {summary} and use them to write an Updated Resume Summary. 
                        Write Only using 1st person pronouns.
                        Don't leave out key info and don't makeup any info on your own:

                        Update Resume Summary:
                        
                        """

recruitor_update_prompt = PromptTemplate(
    input_variables=["skills", "summary"],
    template=recruitor_template
)

In [60]:
recruitor_enhance_chain = LLMChain(llm=MISTRAL_ORCA, prompt=recruitor_update_prompt)

## Chaining Them Together

In [61]:
from langchain.chains import SimpleSequentialChain, SequentialChain

In [62]:
full_chain = SequentialChain(chains=[skill_extract_chain, recruitor_enhance_chain], verbose=True, input_variables=["job_desc", "summary"])

In [63]:
full_chain({'job_desc':jd_description, 'summary': PROFILE_SUMMARY})



> Entering new SequentialChain chain...

To make the Job Description ATS friendly, extract key skills from the job description provided. For example, key skills might include: Python, Machine Learning, Google Cloud Platform (GCP), Spark, SQL, Data Engineering, Big Data, and ETL processes. These are just a few examples; there may be more specific to your job posting.

Key Skills:
1. Proficiency in Google Cloud Platform (GCP) tools including Vertex AI, BigQuery, Dataproc, GKE, and GCS
2. Working experience with native ML orchestration systems such as Kubeflow, Step Functions, MLflow, Airflow, TFX...
3. Experience in ETL processes
4. Proficiency in SQL
5. Familiarity with GIT for version control
6. Good understanding of Machine Learning concepts
7. Experience with object-oriented/object function scripting languages: Python, Java, C++, Scala, etc.
8. Previous experience developing predictive models in a production environment, MLOps and model integration into larger scale applications.
9

{'job_desc': "\nJob Description\n\n    Identify new opportunities to improve business processes and improve consumer experiences, and prototype solutions to demonstrate value with a crawl, walk, run mindset. \n    Work with data scientists and analysts to create and deploy new product features on the ecommerce website, in-store portals and the Levi's mobile app. \n    Implement end-to-end solutions across the full breadth of ML model development lifecycle. The specific role includes working hand in hand with the scientists from the point of data exploration for model development to the point of building features, ml pipelines and deploying them in production. You will have an opportunity to work on both batch and real time models. The role also involves operational support. \n    Establish scalable, efficient, automated processes for data analyses, model development, validation, and implementation. \n    Write efficient and scalable software to ship products in an iterative, continual 